In [1]:
import pypdb
import os
import pandas as pd
import pickle
from pypdb.clients.pdb import pdb_client
import tqdm
from tqdm import tqdm

import gzip
import numpy as np
from Bio.PDB import *
from Bio.PDB.Polypeptide import three_to_one, is_aa

import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
warnings.simplefilter('ignore', PDBConstructionWarning)

In [2]:
## This code for pdb file manipulation is taken is taken from https://github.com/compbiomed-unito/acdc-nn/blob/master/acdc_nn/util.py

def magic_open(path):
    return (gzip.open if path.endswith('.gz') else open)(path, 'rt')

def pdb2seq(pp):
    ''' pdb2seq(pp) takes a pdb_structure_chain 
    and return its sequence '''
    seq = [] # pp.get_sequence()
    reslist = []
    for ppc  in pp:
        reslist += [res for res in ppc]
        seq += [str(ppc.get_sequence())]
    return "".join(seq)

def map_pdb_pos(pp):
    ''' map_pdb_pos
    Returns two dicts seq2pdb[seq_pos], pdb2seq[pdb_pos]'''
    reslist = []
    for ppc  in pp:
        reslist += [res for res in ppc]
    seq2pdb = dict(zip( map(str,range(1,len(reslist)+1)), [str(r.get_id()[1])+r.get_id()[2].strip() for r in reslist]))
    pdb2seq = dict(zip( [str(r.get_id()[1])+r.get_id()[2].strip() for r in reslist], map(str,range(1,len(reslist)+1)) ))
    return seq2pdb, pdb2seq

def pdb2info(pdb_file, chain):
    ''' pdb2info(pdb_file) 
    Returns structure, polypeptide '''
    parser=PDBParser(QUIET=True)
    with magic_open(pdb_file) as f:
        structure = parser.get_structure('X', f)
    pchain=structure[0][chain]
    ppb=PPBuilder()
    pp = ppb.build_peptides(pchain, aa_only=False) #[0]
    return (structure, pchain, pdb2seq(pp), *map_pdb_pos(pp)) 

# S2648

In [3]:
df_S2648 = pd.read_csv('DATA/S2648.csv')

In [4]:
print('Total dataset length', len(df_S2648))
pdb_ids = list(set([t.split()[0].upper() for t in df_S2648.PDB_CHAIN.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 2648
Total number of different chains in dataset 132


In [5]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [6]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

verbatim_pdb_ids = {'1LVEA'}


print('Processing s2648')

for idx in tqdm(range(len(df_S2648))):
    pdb_id = df_S2648.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_S2648.iloc[idx]['WILD_RES']
    pos = str(df_S2648.iloc[idx]['POSITION'])
    mutant_aa = df_S2648.iloc[idx]['MUTANT_RES']
    exp_ddg = df_S2648.iloc[idx]['EXP_DDG']
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id in verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing s2648


 41%|████████████████████████████████████████████████████████████████▋                                                                                             | 1084/2648 [01:31<01:03, 24.70it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 41%|████████████████████████████████████████████████████████████████▊                                                                                             | 1087/2648 [01:31<01:01, 25.43it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

 44%|█████████████████████████████████████████████████████████████████████                                                                                         | 1157/2648 [01:35<01:04, 23.10it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 44%|█████████████████████████████████████████████████████████████████████▎                  

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                           | 2188/2648 [03:03<00:10, 42.87it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 83%|████████████████████████████████████████████████████████████████████████████████████████

In [7]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/S2648.csv')

# S3488

In [8]:
df_1744 = pd.read_csv('DATA/Q1744.txt', sep = ' ', names = ['PDB_CHAIN', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [9]:
print('Total dataset length', len(df_1744))
pdb_ids = list(set([t.split()[0].upper() for t in df_1744.PDB_CHAIN.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 1744
Total number of different chains in dataset 127


In [10]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [11]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

verbatim_pdb_ids = {'1LVEA'}


print('Processing S3488')

for idx in tqdm(range(len(df_1744))):
    pdb_id = df_1744.iloc[idx]['PDB_CHAIN']
    wild_aa = df_1744.iloc[idx]['WILD_RES']
    pos = str(df_1744.iloc[idx]['POSITION'])
    mutant_aa = df_1744.iloc[idx]['MUTANT_RES']
    exp_ddg = df_1744.iloc[idx]['EXP_DDG']
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id in verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(-1*exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))
            
            mut.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            wt.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing S3488


  1%|█▌                                                                                                                                                              | 17/1744 [00:01<02:07, 13.60it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
  1%|█▉                                                                                                                                                              | 21/1744 [00:01<02:07, 13.52it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
  4%|██████▋                                                           

 21%|█████████████████████████████████▏                                                                                                                             | 364/1744 [00:28<01:37, 14.14it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 21%|██████████████████████████████████                                                                                                                             | 373/1744 [00:28<01:18, 17.38it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 23%|████████████████████████████████████▌                                                                                                                          | 401/1744 [00:31<02:56,  7.60it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

 33%|████████████████████████████████████████████████████                                                                                                           | 571/1744 [00:45<01:29, 13.10it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 33%|█████████████████████████████████████████████████████▏                                                                                                         | 583/1744 [00:47<03:16,  5.90it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 34%|█████████████████████████████████████████████████████▉                                                                                                         | 592/1744 [00:47<01:30, 12.75it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

 49%|█████████████████████████████████████████████████████████████████████████████▏                                                                                 | 847/1744 [01:12<00:57, 15.49it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 49%|██████████████████████████████████████████████████████████████████████████████▏                                                                                | 857/1744 [01:12<00:40, 21.96it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 51%|██████████████████████████████████████████████████████████████████

 71%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                             | 1239/1744 [01:51<00:56,  8.89it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 72%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                             | 1247/1744 [01:51<00:47, 10.52it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 72%|██████████████████████████████████████████████████████████████████

 89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 1546/1744 [02:18<00:16, 12.16it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍              | 1583/1744 [02:24<00:33,  4.74it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌             | 1596/1744 [02:26<00:35,  4.21it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

In [12]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/S3488.csv')

# S3421

In [13]:
df_3421 = pd.read_csv('DATA/Q3421.txt', sep = '\t', skiprows = 2, index_col=False,
                      names = ['PDB_ID', 'PDB_CHAIN', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG', 'T', 'PH', 'POS2'])

/tmp/ipykernel_10906/2764013025.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_3421 = pd.read_csv('DATA/Q3421.txt', sep = '\t', skiprows = 2, index_col=False,


In [14]:
print('Total dataset length', len(df_3421))
pdb_ids = list(set([t.split()[0].upper() for t in df_3421.PDB_ID.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 3421
Total number of different chains in dataset 148


In [15]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [16]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

verbatim_pdb_ids = {'1LVEA'}


print('Processing S3421')

for idx in tqdm(range(len(df_3421))):
    pdb_id = df_3421.iloc[idx]['PDB_ID'].upper() + df_3421.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_3421.iloc[idx]['WILD_RES']
    pos = str(df_3421.iloc[idx]['POSITION'])
    mutant_aa = df_3421.iloc[idx]['MUTANT_RES']
    exp_ddg = df_3421.iloc[idx]['EXP_DDG']
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id in verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing S3421


  7%|███████████                                                                                                                                                    | 237/3421 [00:44<01:49, 29.14it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 12%|███████████████████▎                                                                                                                                           | 416/3421 [00:54<03:12, 15.63it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

Error for 1LVEA expected Q at position 89 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

 31%|████████████████████████████████████████████████▊                                                                                                             | 1058/3421 [01:40<03:01, 12.99it/s]

Error for 1LVEA expected S at position 29 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

 44%|████████████████████████████████████████████████████████████████████▊                                                                                         | 1491/3421 [02:18<00:58, 32.83it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 45%|███████████████████████████████████████████████████████████████████████▎                                                                                      | 1543/3421 [02:20<00:58, 31.91it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 45%|██████████████████████████████████████████████████████████████████

 46%|████████████████████████████████████████████████████████████████████████▉                                                                                     | 1579/3421 [02:21<01:04, 28.55it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

 47%|██████████████████████████████████████████████████████████████████████████▋                                                                                   | 1616/3421 [02:22<01:08, 26.43it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                           | 2835/3421 [03:44<00:11, 50.41it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

In [17]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/S3421.csv')

# ACDC-varibench

In [18]:
df_acdc_varibench = pd.concat([pd.read_csv(os.path.join('DATA/varibench/', f), sep = ' ',
            names = ['PDB_CHAIN', 'MUTATION', 'EXP_DDG']) for f in os.listdir('DATA/varibench/')]).drop_duplicates()

In [19]:
print('Total dataset length', len(df_acdc_varibench))
pdb_ids = list(set([t.split()[0].upper() for t in df_acdc_varibench.PDB_CHAIN.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 1387
Total number of different chains in dataset 78


In [20]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [21]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

no_verbatim_pdb_ids = {'1C9OA', '1VQBA'}


print('Processing ACDC-varibench')

for idx in tqdm(range(len(df_acdc_varibench))):
    pdb_id = df_acdc_varibench.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_acdc_varibench.iloc[idx]['MUTATION'][0]
    pos = df_acdc_varibench.iloc[idx]['MUTATION'][1:-1]
    mutant_aa = df_acdc_varibench.iloc[idx]['MUTATION'][-1]
    exp_ddg = df_acdc_varibench.iloc[idx]['EXP_DDG']
    
    #if pdb_id!= '1CLWA':
    #    continue
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id not in no_verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing ACDC-varibench


  1%|█▍                                                                                                                                                              | 13/1387 [00:00<01:45, 12.97it/s]

Error for 1AM7A expected H at position 30 
Sequence is MVEINNQRKAFLDMLAWSEGTDNGRQKTRNHGYDVIVGGELFTDYSDHPRKLVTLNPKLKSTGAGRYQLLSRWWDAYRKQLGLKDFSPKSQDAVALQQIKERGALPMIDRGDIRQAIDRCSNIWASLPGAGYGQFEHKADSLIAKFKEAGGTVR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': 

 15%|███████████████████████▌                                                                                                                                       | 205/1387 [00:07<00:21, 55.69it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 17%|██████████████████████████▍                                                             

Error for 1ONCA expected M at position 22 
Sequence is EDWLTFQKKHITNTRDVDCDNIMSTNLFHCKDKNTFIYSRPEPVKAICKGIIASKNVLTTSEFYLSDCNVTSRPCKYKLKKSTNKFCVTCENQAPVHFVGVGSC
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69', '70': '7

...

 46%|█████████████████████████████████████████████████████████████████████████▌                                                                                     | 642/1387 [00:24<00:17, 42.04it/s]

Error for 1YCCA expected P at position 80 
Sequence is TEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHSGQAEGYSYTDANIKKNVLWDENNMSEYLTNPKKYIPGTKMAFGGLKKEKDRNDLITYLKKACE
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69', '70'

 65%|███████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                       | 904/1387 [00:55<00:25, 18.72it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 65%|███████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                       | 907/1387 [00:55<00:23, 20.73it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

Indexing error for 1BNIA position 108 not present in mapping {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69', '70': '70', '71': '71', '72': '72', '73': '73', '74': '74', '75': '75', '76': '76', '77': '77', '78': '78', '79': '79'

 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                          | 1153/1387 [01:05<00:08, 29.11it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                          | 1157/1387 [01:05<00:08, 27.10it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

In [22]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/ACDC_varibench.csv')

# Ssym

In [23]:
df_ssym = pd.read_csv('DATA/s_sym.txt', sep= ' ', names = ['PDB_ID', '_', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [24]:
print('Total dataset length', len(df_ssym))
pdb_ids = list(set([t.split()[0].upper() for t in df_ssym['PDB_ID'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 342
Total number of different chains in dataset 15


In [25]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [26]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

print('Processing Ssym')

for idx in tqdm(range(len(df_ssym))):
    pdb_id = df_ssym.iloc[idx]['PDB_ID'].upper()
    wild_aa = df_ssym.iloc[idx]['WILD_RES']
    pos = str(df_ssym.iloc[idx]['POSITION'])
    mutant_aa = df_ssym.iloc[idx]['MUTANT_RES']
    exp_ddg = df_ssym.iloc[idx]['EXP_DDG']
    
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #  seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing Ssym


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 342/342 [00:15<00:00, 22.37it/s]


In [27]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/ssym.csv')

In [28]:
pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss}).to_csv('DATASETS/ssym_r.csv')

# Ssym+

In [29]:
df_ssym_plus = pd.read_csv('DATA/Ssym+_experimental.csv', sep= ',')[['Protein', 'DDG', 'Mut_pdb']]

In [30]:
print('Total dataset length', len(df_ssym_plus))
pdb_ids = list(set([t.split()[0].upper() for t in df_ssym_plus['Protein'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 352
Total number of different chains in dataset 19


In [31]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [32]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

print('Processing Ssym+')

for idx in tqdm(range(len(df_ssym_plus))):
    pdb_id = df_ssym_plus.iloc[idx]['Protein'].upper()
    wild_aa = df_ssym_plus.iloc[idx]['Mut_pdb'][0]
    pos = df_ssym_plus.iloc[idx]['Mut_pdb'][1:-1]
    mutant_aa = df_ssym_plus.iloc[idx]['Mut_pdb'][-1]
    exp_ddg = df_ssym_plus.iloc[idx]['DDG']
    
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #  seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing Ssym+


 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋           | 327/352 [00:15<00:00, 26.78it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████          | 330/352 [00:15<00:01, 21.51it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

In [33]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/ssym_plus.csv')

In [34]:
pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss}).to_csv('DATASETS/ssym_plus_r.csv')

# Myoglobin

In [35]:
df_myoglobin = pd.read_csv('DATA/myoglobin.txt', sep= ' ', names = ['PDB_ID', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [36]:
print('Total dataset length', len(df_myoglobin))
pdb_ids = list(set([t.split()[0].upper() for t in df_myoglobin['PDB_ID'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 134
Total number of different chains in dataset 1


In [37]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [38]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

#no_verbatim_pdb_ids = {'1C9OA', '1VQBA'}


print('Processing myoglobin')

for idx in tqdm(range(len(df_myoglobin))):
    pdb_id = df_myoglobin.iloc[idx]['PDB_ID'].upper()
    wild_aa = df_myoglobin.iloc[idx]['WILD_RES']
    pos = str(df_myoglobin.iloc[idx]['POSITION'])
    mutant_aa = df_myoglobin.iloc[idx]['MUTANT_RES']
    exp_ddg = df_myoglobin.iloc[idx]['EXP_DDG']
    
    #if pdb_id!= '1CLWA':
    #    continue
        
    
    pdb = PDBParser().get_structure(pdb_id[:4], f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #  seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing myoglobin


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:13<00:00, 10.03it/s]


In [39]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/myoglobin.csv')

In [40]:
pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss}).to_csv('DATASETS/myoglobin_r.csv')

# P53

In [41]:
df_p53 = pd.read_csv('DATA/p53.txt', sep= ' ', names = ['PDB_ID', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [42]:
print('Total dataset length', len(df_p53))
pdb_ids = list(set([t.split()[0].upper() for t in df_p53['PDB_ID'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 42
Total number of different chains in dataset 1


In [43]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [44]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

print('Processing p53')

for idx in tqdm(range(len(df_p53))):
    pdb_id = df_p53.iloc[idx]['PDB_ID'].upper()
    wild_aa = df_p53.iloc[idx]['WILD_RES']
    pos = str(df_p53.iloc[idx]['POSITION'])
    mutant_aa = df_p53.iloc[idx]['MUTANT_RES']
    exp_ddg = df_p53.iloc[idx]['EXP_DDG']
    
        
    
    pdb = PDBParser().get_structure(pdb_id[:4], f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing p53


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:08<00:00,  4.76it/s]


In [45]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/p53.csv')

# S669

In [46]:
df_p53 = pd.read_csv('DATA/p53.txt', sep= ' ', names = ['PDB_ID', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [47]:
df_S669 = pd.read_csv('DATA/Data_s669_with_predictions.csv')

In [48]:
print('Total dataset length', len(df_S669))
pdb_ids = list(set([t.split()[0][:4].upper() for t in df_S669['Protein'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 669
Total number of different chains in dataset 94


In [49]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [50]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

print('Processing S669')

for idx in tqdm(range(len(df_S669))):
    pdb_id = df_S669.iloc[idx]['Protein'].upper()
    wild_aa = df_S669.iloc[idx]['PDB_Mut'][0]
    pos = df_S669.iloc[idx]['PDB_Mut'][1:-1]
    mutant_aa = df_S669.iloc[idx]['PDB_Mut'][-1]
    exp_ddg = df_S669.iloc[idx]['DDG_checked_dir']
    
        
    
    pdb = PDBParser().get_structure(pdb_id[:4], f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing S669


 20%|███████████████████████████████▊                                                                                                                                | 133/669 [00:18<00:41, 12.88it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 20%|████████████████████████████████▎                                                                                                                               | 135/669 [00:18<00:44, 12.05it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

 24%|██████████████████████████████████████▉                                                                                                                         | 163/669 [00:20<00:42, 11.92it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 25%|███████████████████████████████████████▍                                                                                                                        | 165/669 [00:20<00:44, 11.45it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

In [51]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/s669.csv')

In [52]:
pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss}).to_csv('DATASETS/s669_r.csv')

# PROSTATA dataset

In [53]:
df_article_dataset = pd.read_pickle('DATA/dataset_our_w_clusters_v2.5.pkl')

In [54]:
print('Total dataset length', len(df_article_dataset))
pdb_ids = list(set([t.split()[0].upper() for t in df_article_dataset['pdb_id'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 6547
Total number of different chains in dataset 663


In [55]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [56]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []
ssym_splits = []
s669_splits = []
clusters = []

print('Processing article dataset')

for idx in tqdm(range(len(df_article_dataset))):
    pdb_id = df_article_dataset.iloc[idx]['pdb_id'].upper() + df_article_dataset.iloc[idx]['pdb_chain'].upper()
    wild_aa = df_article_dataset.iloc[idx]['Mut_code'][0]
    pos = df_article_dataset.iloc[idx]['Mut_code'][1:-1]
    mutant_aa = df_article_dataset.iloc[idx]['Mut_code'][-1]
    exp_ddg = df_article_dataset.iloc[idx]['mean_ddG']
    
    ssym_split = df_article_dataset.iloc[idx]['ssym_split']
    s669_split = df_article_dataset.iloc[idx]['s669_split']
    cluster = df_article_dataset.iloc[idx]['cluster_id'] 
    
        
    
    pdb = PDBParser().get_structure(pdb_id[:4], f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))
            ssym_splits.append(ssym_split)
            s669_splits.append(s669_split)
            clusters.append(cluster)

Processing article dataset


  4%|█████▉                                                                                                                                                         | 246/6547 [01:37<16:02,  6.55it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
  5%|███████▌                                                                                                                                                       | 311/6547 [01:41<07:01, 14.79it/s]

Error for 1AYEA expected A at position 31 
Sequence is LETFVGDQVLEIVPSNEEQIKNLLQLEAQEHLQLDFWKSPTTPGETAHVRVPFVNVQAVKVFLESQGIAYSIMIEDVQVLLDKENEEMLFNRRRERSGNFNFGAYHTLEEISQEMDNLVAEHPGLVSKVNIGSSFENRPMNVLKFSTGGDKPAIWLDAGIHAREWVTQATALWTANKIVSDYGKDPSITSILDALDIFLLPVTNPDGYVFSQTKNRMWRKTRSKVSGSLCVGVDPNRNWDAGFGGPGASSNPCSDSYHGPSANSEVEVKSIVDFIKSHGKVKAFIILHSYSQLLMFPYGYKCTKLDDFDELSEVAQKAAQSLRSLHGTKYKVGPICSVIYQASGGSIDWSYDYGIKYSFAFELRDTGRYGFLLPARQILPTAEETWLGLKAIMEHVRDHPY
Mapping is {'4A': '1', '5A': '2', '6A': '3', '7A': '4', '8A': '5', '9A': '6', '10A': '7', '11A': '8', '12A': '9', '13A': '10', '14A': '11', '15A': '12', '16A': '13', '17A': '14', '18A': '15', '19A': '16', '20A': '17', '21A': '18', '22A': '19', '23A': '20', '24A': '21', '25A': '22', '26A': '23', '27A': '24', '28A': '25', '29A': '26', '30A': '27', '31A': '28', '32A': '29', '33A': '30', '34A': '31', '34B': '32', '34C': '33', '35A': '34', '36A': '35', '37A': '36', '38A': '37', '39A': '38', '40A': '39', '41A': '40', '42A': '41', '47A': '42', 

  5%|███████▋                                                                                                                                                       | 315/6547 [01:41<09:53, 10.50it/s]

Error for 1AYEA expected E at position 20 
Sequence is LETFVGDQVLEIVPSNEEQIKNLLQLEAQEHLQLDFWKSPTTPGETAHVRVPFVNVQAVKVFLESQGIAYSIMIEDVQVLLDKENEEMLFNRRRERSGNFNFGAYHTLEEISQEMDNLVAEHPGLVSKVNIGSSFENRPMNVLKFSTGGDKPAIWLDAGIHAREWVTQATALWTANKIVSDYGKDPSITSILDALDIFLLPVTNPDGYVFSQTKNRMWRKTRSKVSGSLCVGVDPNRNWDAGFGGPGASSNPCSDSYHGPSANSEVEVKSIVDFIKSHGKVKAFIILHSYSQLLMFPYGYKCTKLDDFDELSEVAQKAAQSLRSLHGTKYKVGPICSVIYQASGGSIDWSYDYGIKYSFAFELRDTGRYGFLLPARQILPTAEETWLGLKAIMEHVRDHPY
Mapping is {'4A': '1', '5A': '2', '6A': '3', '7A': '4', '8A': '5', '9A': '6', '10A': '7', '11A': '8', '12A': '9', '13A': '10', '14A': '11', '15A': '12', '16A': '13', '17A': '14', '18A': '15', '19A': '16', '20A': '17', '21A': '18', '22A': '19', '23A': '20', '24A': '21', '25A': '22', '26A': '23', '27A': '24', '28A': '25', '29A': '26', '30A': '27', '31A': '28', '32A': '29', '33A': '30', '34A': '31', '34B': '32', '34C': '33', '35A': '34', '36A': '35', '37A': '36', '38A': '37', '39A': '38', '40A': '39', '41A': '40', '42A': '41', '47A': '42', 

...

 50%|██████████████████████████████████████████████████████████████████████████████▌                                                                               | 3255/6547 [07:44<03:33, 15.40it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 50%|██████████████████████████████████████████████████████████████████████████████▌                                                                               | 3257/6547 [07:44<04:04, 13.46it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

 50%|███████████████████████████████████████████████████████████████████████████████▎                                                                              | 3287/6547 [07:46<03:28, 15.63it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 50%|███████████████████████████████████████████████████████████████████████████████▎                                                                              | 3289/6547 [07:47<03:59, 13.62it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

 74%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                        | 4866/6547 [11:16<01:25, 19.56it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 74%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                        | 4869/6547 [11:16<01:19, 21.02it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 74%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                        | 4872/6547 [11:16<01:30, 18.50it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 6085/6547 [12:55<00:40, 11.47it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉           | 6088/6547 [12:56<00:39, 11.73it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

 93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌          | 6116/6547 [12:58<00:34, 12.58it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌       | 6238/6547 [14:07<00:37,  8.34it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

In [57]:
article_dataset = pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss,
              'ssym_split': ssym_splits,
              's669_split': s669_splits,
              'cluster': clusters})

In [58]:
article_dataset_rev = pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss,
              'ssym_split': ssym_splits,
              's669_split': s669_splits,
              'cluster': clusters})

In [59]:
article_dataset_with_rev = pd.concat([article_dataset, article_dataset_rev]).drop_duplicates(['wt_seq', 'mut_seq'])
print(len(article_dataset_with_rev))
#10542

10542


In [60]:
article_dataset_with_rev[article_dataset_with_rev.ssym_split == 'train'][['wt_seq', 'mut_seq', 'ddg', 'pdb_id', 'mut_info', 'pos']].to_csv('DATASETS/new_ds_for_ssym.csv')
article_dataset_with_rev[article_dataset_with_rev.s669_split == 'train'][['wt_seq', 'mut_seq', 'ddg', 'pdb_id', 'mut_info', 'pos']].to_csv('DATASETS/new_ds_for_s669.csv')
article_dataset_with_rev[(article_dataset_with_rev.ssym_split == 'train') & ~(article_dataset_with_rev.pdb_id.str.startswith('1ARR') | article_dataset_with_rev.pdb_id.str.startswith('1N0J'))][['wt_seq', 'mut_seq', 'ddg', 'pdb_id', 'mut_info', 'pos']].to_csv('DATASETS/new_ds_for_ssym_plus.csv')

In [61]:
from sklearn.model_selection import GroupKFold
group_kfold = GroupKFold(n_splits=5)
article_dataset_with_rev = article_dataset_with_rev.reset_index(drop = True)
article_dataset_with_rev['fold'] = None
for fold, (train_index, test_index) in enumerate(group_kfold.split(article_dataset_with_rev.cluster, 
                                                                   article_dataset_with_rev.cluster, 
                                                                   article_dataset_with_rev.cluster)):
    article_dataset_with_rev.loc[test_index, 'fold'] = fold

In [62]:
article_dataset_with_rev[['wt_seq', 'mut_seq', 'ddg', 'pdb_id', 'mut_info', 'pos', 'fold']].to_csv('DATASETS/new_ds_with_folds.csv')

# Case studies datasets

In [63]:
dimer_clusters = {'cluster_162_ref_1AV1_A', 'cluster_4_ref_1BFM_A', 'cluster_161_ref_1ARR_A', 'cluster_34_ref_1R6R_A', 'cluster_139_ref_1SAK_A', 'cluster_222_ref_1ZNJ_B', 'cluster_223_ref_1ZNJ_A', 'cluster_210_ref_1UWO_A', 'cluster_69_ref_3MON_B', 'cluster_47_ref_2KJ3_A', 'cluster_183_ref_1CDC_B', 'cluster_140_ref_1SCE_A'}
gem_clusters = {'cluster_2_ref_1A7V_A', 'cluster_189_ref_1CYC_A', 'cluster_217_ref_1YCC_A', 'cluster_92_ref_1I5T_A', 'cluster_257_ref_451C_A', 'cluster_168_ref_1B5M_A', 'cluster_190_ref_1CYO_A', 'cluster_179_ref_1C52_A', 'cluster_178_ref_1C2R_A', 'cluster_218_ref_1YEA_A', 'cluster_177_ref_1BVC_A'}
transmembrane_clusters = {'cluster_145_ref_1THQ_A', 'cluster_200_ref_1FEP_A', 'cluster_230_ref_2BRD_A', 'cluster_129_ref_1QJP_A'}
        

In [64]:
article_dataset_full = article_dataset_with_rev[['wt_seq', 'mut_seq', 'ddg', 'pdb_id', 'mut_info', 'pos', 'cluster']]

In [65]:
article_dataset_full[~article_dataset_full.cluster.isin(dimer_clusters)].to_csv('DATASETS/case_study_dimer_train.csv')
article_dataset_full[article_dataset_full.cluster.isin(dimer_clusters)].to_csv('DATASETS/case_study_dimer_test.csv')

In [66]:
article_dataset_full[~article_dataset_full.cluster.isin(gem_clusters)].to_csv('DATASETS/case_study_gem_train.csv')
article_dataset_full[article_dataset_full.cluster.isin(gem_clusters)].to_csv('DATASETS/case_study_gem_test.csv')

In [67]:
article_dataset_full[~article_dataset_full.cluster.isin(transmembrane_clusters)].to_csv('DATASETS/case_study_transmembrane_train.csv')
article_dataset_full[article_dataset_full.cluster.isin(transmembrane_clusters)].to_csv('DATASETS/case_study_transmembrane_test.csv')

# Ssym less strict train set filtering

In [68]:
df_ssym_plus = pd.concat([pd.read_csv('DATASETS/ssym_plus.csv'), pd.read_csv('DATASETS/ssym_plus_r.csv')])
df_ssym_plus['exact'] = True

In [69]:
article_dataset_full = article_dataset_full.merge(df_ssym_plus[['wt_seq', 'mut_seq', 'mut_info', 'exact']], left_on = ['wt_seq', 'mut_seq', 'mut_info'], 
                           right_on = ['wt_seq', 'mut_seq', 'mut_info'], 
                           how = 'left')

In [70]:
article_dataset_full[article_dataset_full.exact.isna()][['wt_seq', 'mut_seq', 'ddg', 'pdb_id', 'mut_info', 'pos']].to_csv('DATASETS/new_ds_for_ssym_plus_less_strict.csv')